In [1]:
#Keep in mind this is by default set to sample every 4ms 
#To change it you have to change the rate in the ESP32code 
#Working on a way to set sample rate in code 
#Also there is no way to restart timer without manually pushing reboot yet sorry!

#Imports
import socket, time

ESP_IP   = "10.0.0.103"   # set to ESP32's IP
ESP_PORT = 9000          # must match your sketch
RETRY_SEC = 1.5          # wait before reconnecting

In [2]:
def stream_samples(host: str, port: int):
    """
    Generator that yields (t_ms:int, value:float) from 't_ms,value' lines.
    Auto-reconnects on errors/disconnects. 
    """
    buf = b""
    while True:
        s = None
        try:
            s = socket.create_connection((host, port), timeout=5)
            s.settimeout(None)  # blocking
            print(f"[connected] {host}:{port}")
            buf = b""
            while True:
                chunk = s.recv(4096)
                if not chunk:
                    raise ConnectionError("server closed connection")
                buf += chunk
                while b"\n" in buf:
                    line, buf = buf.split(b"\n", 1)
                    line = line.strip()
                    if not line or line.startswith(b"#"):
                        continue
                    try:
                        t_ms_b, val_b = line.split(b",", 1)
                        yield int(t_ms_b), float(val_b)
                    except Exception:
                        # skip malformed lines
                        pass
        except KeyboardInterrupt:
            # clean disconnect then bubble up to exit
            try:
                if s: s.shutdown(socket.SHUT_RDWR)
            except:
                pass
            if s: s.close()
            print("\n[bye]")
            raise
        except Exception as e:
            print(f"[reconnect] {e}; retrying in {RETRY_SEC}s…")
            try:
                if s: s.shutdown(socket.SHUT_RDWR)
            except:
                pass
            if s: s.close()
            time.sleep(RETRY_SEC)

In [ ]:
if __name__ == "__main__":
    try:
        for t_ms, val in stream_samples(ESP_IP, ESP_PORT): #t_ms i time in ms and val is voltage in uv
            # Put whatever you code is here
                print(f"{t_ms},{val}") 
            
    except KeyboardInterrupt:
        pass


[reconnect] timed out; retrying in 1.5s…
[reconnect] timed out; retrying in 1.5s…
[reconnect] timed out; retrying in 1.5s…
[reconnect] timed out; retrying in 1.5s…
[connected] 10.0.0.103:9000
4,12.43
8,24.09
12,34.23
16,42.22
20,47.55
24,49.9
28,49.11
32,45.24
36,38.53
40,29.39
44,18.41
48,6.27
52,-6.27
56,-18.41
60,-29.39
64,-38.53
68,-45.24
72,-49.11
76,-49.9
80,-47.55
84,-42.22
88,-34.23
92,-24.09
96,-12.43
100,0.0
104,12.43
108,24.09
112,34.23
116,42.22
120,47.55
124,49.9
128,49.11
132,45.24
136,38.53
140,29.39
144,18.41
148,6.27
152,-6.27
156,-18.41
160,-29.39
164,-38.53
168,-45.24
172,-49.11
176,-49.9
180,-47.55
184,-42.22
188,-34.23
192,-24.09
196,-12.43
200,0.0
204,12.43
208,24.09
212,34.23
216,42.22
220,47.55
224,49.9
228,49.11
232,45.24
236,38.53
240,29.39
244,18.41
248,6.27
252,-6.27
256,-18.41
260,-29.39
264,-38.53
268,-45.24
272,-49.11
276,-49.9
280,-47.55
284,-42.22
288,-34.23
292,-24.09
296,-12.43
300,0.0
304,12.43
308,24.09
312,34.23
316,42.22
320,47.55
324,49.9
328,49.